In [ ]:
#========================== نصب کتابخانه‌های مورد نیاز
!pip install -q transformers datasets accelerate peft bitsandbytes rouge-score torch



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#========================== وارد کردن کتابخانه‌های مورد نیاز
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset
from rouge_score import rouge_scorer
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(" کتابخانه‌ها با موفقیت وارد شدند")


 کتابخانه‌ها با موفقیت وارد شدند


In [3]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

#========================== نمایش مسیر cache برای مدل‌ها
import os
from huggingface_hub import cached_assets_path

# مسیر پیش‌فرض cache در ویندوز
cache_dir = os.path.expanduser("~/.cache/huggingface/hub")
print(f" مسیر cache مدل‌ها: {cache_dir}")
print(f" مدل از Hugging Face Hub دانلود می‌شود: https://huggingface.co/{MODEL_NAME}")

#========================== تنظیمات 4-bit quantization برای کاهش مصرف RAM و GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f" در حال بارگذاری مدل {MODEL_NAME}...")
print(" اولین بار ممکن است چند دقیقه طول بکشد (دانلود ~2.2GB)")

#========================== بارگذاری توکنایزر
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#============================= بارگذاری مدل با quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

#=========================== آماده‌سازی مدل برای آموزش با QLoRA
model = prepare_model_for_kbit_training(model)

print(f" مدل با موفقیت بارگذاری شد!")
print(f" تعداد پارامترهای مدل: {model.num_parameters():,}")

# نمایش مسیر دقیق ذخیره‌سازی مدل
model_path = os.path.join(cache_dir, f"models--{MODEL_NAME.replace('/', '--')}")
if os.path.exists(model_path):
    print(f" مدل در این مسیر ذخیره شده: {model_path}")


📥 در حال بارگذاری مدل TinyLlama/TinyLlama-1.1B-Chat-v1.0...


tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ مدل با موفقیت بارگذاری شد!
📊 تعداد پارامترهای مدل: 1,100,048,384


In [ ]:
#========================== تنظیمات برای کاهش زمان آموزش (اختیاری)
# اگر می‌خواهید سریع‌تر تست کنید، این مقادیر را کاهش دهید

# برای تست سریع (حدود 1-2 ساعت روی CPU):
# MAX_TRAIN_SAMPLES = 1000
# MAX_VAL_SAMPLES = 100
# NUM_EPOCHS = 1

# برای آموزش کامل (9-15 ساعت روی CPU):
MAX_TRAIN_SAMPLES = 5000
MAX_VAL_SAMPLES = 500
NUM_EPOCHS = 3

print("⚠️ توجه: بدون GPU، آموزش روی CPU خیلی کند است (9-15 ساعت)")
print("💡 توصیه: از Google Colab با GPU رایگان استفاده کنید")
print(f"📊 تنظیمات فعلی: {MAX_TRAIN_SAMPLES} نمونه آموزشی، {NUM_EPOCHS} epoch")



In [ ]:
#========================== بارگذاری دیتاست CNN/DailyMail
print(" در حال بارگذاری دیتاست ")

dataset = load_dataset("cnn_dailymail", "3.0.0")

#========================== نمایش اطلاعات دیتاست
print(f" اطلاعات دیتاست:")
print(f"  - Train: {len(dataset['train'])} نمونه")
print(f"  - Validation: {len(dataset['validation'])} نمونه")
print(f"  - Test: {len(dataset['test'])} نمونه")

#========================== نمایش یک نمونه
print("\n نمونه از دیتاست:")
print(f"Article (اولین 200 کاراکتر): {dataset['train'][0]['article'][:200]}...")
print(f"Highlights: {dataset['train'][0]['highlights']}")


# استفاده از متغیرهای تعریف شده در سلول قبل (یا مقادیر پیش‌فرض)
if 'MAX_TRAIN_SAMPLES' not in globals():
    MAX_TRAIN_SAMPLES = 5000
if 'MAX_VAL_SAMPLES' not in globals():
    MAX_VAL_SAMPLES = 500

train_dataset = dataset['train'].select(range(min(MAX_TRAIN_SAMPLES, len(dataset['train']))))
val_dataset = dataset['validation'].select(range(min(MAX_VAL_SAMPLES, len(dataset['validation']))))

print(f" دیتاست آماده شد!")
print(f"  - نمونه‌های آموزشی: {len(train_dataset)}")
print(f"  - نمونه‌های اعتبارسنجی: {len(val_dataset)}")


 در حال بارگذاری دیتاست 
 اطلاعات دیتاست:
  - Train: 287113 نمونه
  - Validation: 13368 نمونه
  - Test: 11490 نمونه

 نمونه از دیتاست:
Article (اولین 200 کاراکتر): LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on ...
Highlights: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .
 دیتاست آماده شد!
  - نمونه‌های آموزشی: 5000
  - نمونه‌های اعتبارسنجی: 500


In [ ]:
#========================== تابع برای فرمت‌دهی ورودی/خروجی
def format_prompt(article, highlights=None):
    """
    فرمت‌دهی ورودی برای مدل
    ورودی: article (مقاله)
    خروجی: highlights (خلاصه)
    """
    prompt = f"""### Article:
{article}

### Summary:
"""
    
    if highlights:
        prompt += highlights
    
    return prompt

#========================== تابع برای پردازش داده‌ها
def preprocess_function(examples):
    """پردازش و توکنایز کردن داده‌ها"""
    #========================== ترکیب ورودی و خروجی
    full_texts = [format_prompt(article, highlights) for article, highlights in zip(examples['article'], examples['highlights'])]
    
    #========================== توکنایز کردن
    model_inputs = tokenizer(
        full_texts,
        max_length=1024,
        truncation=True,
        padding=False,
    )
    

    labels = []
    for i, text in enumerate(full_texts):
        #========================== پیدا کردن موقعیت شروع summary در متن اصلی
        summary_start = text.find("### Summary:")
        if summary_start != -1:
            #========================== توکنایز کردن فقط بخش prompt (قبل از summary)
            prompt_text = text[:summary_start + len("### Summary:\n")]
            prompt_tokens = tokenizer(prompt_text, return_tensors="pt", max_length=1024, truncation=True)
            prompt_length = len(prompt_tokens['input_ids'][0])
            
            #========================== ایجاد labels: -100 برای prompt، و input_ids برای summary
            input_ids = model_inputs['input_ids'][i]
            #========================== اطمینان از اینکه prompt_length از input_ids تجاوز نکند
            prompt_length = min(prompt_length, len(input_ids))
            label = [-100] * prompt_length
            # input_ids در حالت batched=True یک لیست است، نه tensor
            label.extend(input_ids[prompt_length:])
            
            #========================== اطمینان از هماهنگی طول
            if len(label) < len(input_ids):
                label.extend([-100] * (len(input_ids) - len(label)))
            elif len(label) > len(input_ids):
                label = label[:len(input_ids)]
            
            labels.append(label)
        else:
            labels.append([-100] * len(model_inputs['input_ids'][i]))
    
    model_inputs['labels'] = labels
    return model_inputs

#========================== پردازش داده‌ها
print(" در حال پردازش داده‌های آموزشی...")
train_tokenized = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train data"
)

print(" در حال پردازش داده‌های اعتبارسنجی...")
val_tokenized = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names,
    desc="Tokenizing validation data"
)

print(" داده‌ها پردازش شدند!")


 در حال پردازش داده‌های آموزشی...


Tokenizing train data:   0%|          | 0/5000 [00:00<?, ? examples/s]

 در حال پردازش داده‌های اعتبارسنجی...


Tokenizing validation data:   0%|          | 0/500 [00:00<?, ? examples/s]

 داده‌ها پردازش شدند!


In [9]:
#========================== تنظیم QLoRA با PEFT
print(" در حال تنظیم QLoRA...")

lora_config = LoraConfig(
    r=16,  #========================== rank - هرچه بیشتر باشد، پارامترهای بیشتری آموزش می‌بینند
    lora_alpha=32,  #========================== scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # لایه‌های هدف
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

#========================== اعمال LoRA به مدل
model = get_peft_model(model, lora_config)

#========================== نمایش پارامترهای قابل آموزش
model.print_trainable_parameters()

print(" QLoRA تنظیم شد!")


 در حال تنظیم QLoRA...
trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338
 QLoRA تنظیم شد!


In [ ]:
#========================== Import اضافی در صورت نیاز
from transformers import DataCollatorForSeq2Seq

#========================== تنظیمات آموزش
# استفاده از NUM_EPOCHS از سلول قبل (یا مقدار پیش‌فرض)
if 'NUM_EPOCHS' not in globals():
    NUM_EPOCHS = 3

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

#========================== Data collator
# استفاده از DataCollatorForSeq2Seq برای summarization که با labels از قبل تنظیم شده بهتر کار می‌کند
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
)

#========================== ایجاد Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

print(" Trainer آماده شد!")
print(" شروع آموزش...")

#========================== شروع آموزش
trainer.train()

print(" آموزش با موفقیت به پایان رسید!")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


 Trainer آماده شد!
 شروع آموزش...


In [ ]:
#========================== ذخیره مدل آموزش دیده
import os

#========================== مسیر ذخیره مدل فاین‌تیون شده
fine_tuned_model_path = "./fine_tuned_model"
fine_tuned_model_abs_path = os.path.abspath(fine_tuned_model_path)

print(" در حال ذخیره مدل...")
print(f" مسیر ذخیره: {fine_tuned_model_abs_path}")

trainer.save_model(fine_tuned_model_path)
tokenizer.save_pretrained(fine_tuned_model_path)

print(" مدل ذخیره شد!")
print(f" مدل فاین‌تیون شده در این مسیر ذخیره شده: {fine_tuned_model_abs_path}")
print(f"   - فایل‌های LoRA adapter در این پوشه هستند")
print(f"   - برای استفاده مجدد، باید مدل پایه + adapter را بارگذاری کنید")


#========================== تابع برای تولید خلاصه
def generate_summary(article, max_length=200):
    """تولید خلاصه از یک مقاله"""
    prompt = format_prompt(article)
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            num_beams=4,
            early_stopping=True,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    #========================== استخراج فقط بخش summary
    if "### Summary:" in generated_text:
        summary = generated_text.split("### Summary:")[-1].strip()
    else:
        summary = generated_text
    
    return summary

#========================== تست روی چند نمونه
print("\n نمونه‌های تولید شده:\n")
print("=" * 80)

test_samples = val_dataset.select(range(5))

for i, sample in enumerate(test_samples):
    print(f"\n{'='*80}")
    print(f"نمونه {i+1}")
    print(f"{'='*80}")
    
    article = sample['article'][:500]  #========================== فقط 500 کاراکتر اول برای نمایش
    true_highlights = sample['highlights']
    
    print(f"\n مقاله (500 کاراکتر اول):\n{article}...")
    print(f"\n خلاصه واقعی:\n{true_highlights}")
    
    generated_summary = generate_summary(sample['article'])
    print(f"\n خلاصه تولید شده:\n{generated_summary}")
    print("\n" + "-"*80)


In [ ]:
#========================== ارزیابی با معیار ROUGE
print(" در حال ارزیابی با معیار ROUGE...")

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

#========================== ارزیابی روی نمونه‌های تست
test_samples = val_dataset.select(range(min(100, len(val_dataset))))

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

print(f"در حال ارزیابی {len(test_samples)} نمونه...")

for sample in tqdm(test_samples):
    true_highlights = sample['highlights']
    generated_summary = generate_summary(sample['article'])
    
    scores = scorer.score(true_highlights, generated_summary)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

#========================== محاسبه میانگین
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

print("\n" + "="*80)
print(" نتایج ارزیابی ROUGE:")
print("="*80)
print(f"ROUGE-1 F1: {avg_rouge1:.4f}")
print(f"ROUGE-2 F1: {avg_rouge2:.4f}")
print(f"ROUGE-L F1: {avg_rougeL:.4f}")
print("="*80)

#========================== نمایش توزیع نمرات
print(f"\n آمار نمرات:")
print(f"ROUGE-1: میانگین={avg_rouge1:.4f}, انحراف معیار={np.std(rouge1_scores):.4f}")
print(f"ROUGE-2: میانگین={avg_rouge2:.4f}, انحراف معیار={np.std(rouge2_scores):.4f}")
print(f"ROUGE-L: میانگین={avg_rougeL:.4f}, انحراف معیار={np.std(rougeL_scores):.4f}")


In [ ]:
#========================== بررسی انسانی - نمایش چند نمونه با جزئیات بیشتر
print("🔍 بررسی انسانی - نمونه‌های منتخب:\n")
print("="*80)

#========================== انتخاب چند نمونه تصادفی برای بررسی دقیق‌تر
import random
random.seed(42)
selected_indices = random.sample(range(len(test_samples)), min(5, len(test_samples)))

detailed_results = []

for idx in selected_indices:
    sample = test_samples[idx]
    article = sample['article']
    true_highlights = sample['highlights']
    generated_summary = generate_summary(article)
    
    #========================== محاسبه ROUGE برای این نمونه
    scores = scorer.score(true_highlights, generated_summary)
    
    detailed_results.append({
        'article_length': len(article),
        'true_summary_length': len(true_highlights),
        'generated_summary_length': len(generated_summary),
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure,
    })
    
    print(f"\n{'='*80}")
    print(f"نمونه {idx + 1}")
    print(f"{'='*80}")
    print(f"\n مقاله (اولین 300 کاراکتر):\n{article[:300]}...")
    print(f"\n خلاصه واقعی ({len(true_highlights)} کاراکتر):\n{true_highlights}")
    print(f"\n خلاصه تولید شده ({len(generated_summary)} کاراکتر):\n{generated_summary}")
    print(f"\n نمرات ROUGE:")
    print(f"  ROUGE-1: {scores['rouge1'].fmeasure:.4f}")
    print(f"  ROUGE-2: {scores['rouge2'].fmeasure:.4f}")
    print(f"  ROUGE-L: {scores['rougeL'].fmeasure:.4f}")
    print("\n" + "-"*80)

#========================== ایجاد DataFrame برای تحلیل
df_results = pd.DataFrame(detailed_results)
print("\n خلاصه نتایج نمونه‌های بررسی شده:")
print(df_results.describe())
